# Team Ernst Engel Nutritional Content Notebook
In this notebook, we obtain the nutritional contents for the foods of interest for our project.

In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, widgets
import fooddatacentral as fdc

# This is a local file unique to Jordan's computer
# It contains his USDA FoodData Central API Key
# Do Not Uncomment or it will cause an error
import config

In [48]:
foods_arr = np.array(pd.read_csv('./data/foods_oi.csv').iloc[:10, 0])
apikey = config.API_KEY

In [52]:
D = {}
for food in foods_arr:
    try:
        FDC = df.loc[df.index==food,:].FDC.values[0]
        D[food] = fdc.nutrients(apikey,FDC).Quantity
        print(food)
    except:
        pass

D = pd.DataFrame(D,dtype=float)

In [56]:
foods_arr

array(['Avocado', 'Beans (dry)', 'Beans (fresh)', 'Beef',
       'Beef (roasted)', 'Beef Liver', 'Beef Offal', 'Biscuits', 'Bogoya',
       'Bongo'], dtype=object)

In [67]:
fdc.nutrients(apikey, '2485214')

,Quantity,Units
"Sodium, Na",40.00,mg
"Vitamin D (D2 + D3), International Units",42.00,IU
Protein,3.33,g
Total Sugars,5.00,g
Energy,54.00,kcal
"Vitamin A, IU",2500.00,IU
Cholesterol,8.00,mg
"Fatty acids, total saturated",1.04,g
"Fatty acids, total trans",0.00,g
"Fiber, total dietary",0.00,g
